# Sigma: CodeBotix Data Consolidation

#### Steps Taken
1. Corrected mispalced columns in '2 Day learning surveys 2024-2025 American center.xlsx' Sheet 2
2. 

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import torch

#### Utils

In [2]:
def InList(elem, l:list) -> bool:
  for item in l:
    if (elem.lower() in item.lower()) or (item.lower() in elem.lower()):
      return True
      break
  return False

def columnLike(X, cols: list, singleOp=True) -> str:
  if not isinstance(X, list): X = [X]
  op = []
  for xi in X:
    for item in cols:
      if ((xi.lower().strip() in item.lower().strip()) or (item.lower().strip() in xi.lower().strip())) and ('unnamed' not in item.lower().strip()):
        op.append(item)
  if len(op) == 0: return None
  if (len(op) == 1) or singleOp: op = op[0]
  return op

#### Getting the Data

In [3]:
dataSets = []

dataSets.append(pd.read_excel('data/1 hour Online intro to Robotics workshop-pilot.xlsx', sheet_name=0)) #0, 2, 3, 5, 9, 11
dataSets.append(pd.read_excel('data/1 hour Online intro to Robotics workshop-pilot.xlsx', sheet_name=1))
dataSets.append(pd.read_excel('data/10 day Summercamp Anna library Feedback .xlsx', sheet_name=0))
dataSets.append(pd.read_excel('data/2 Day learning surveys 2024-2025 American center.xlsx', sheet_name=0))
dataSets.append(pd.read_excel('data/2 Day learning surveys 2024-2025 American center.xlsx', sheet_name=1))
dataSets.append(pd.read_excel('data/3 days  Robotics Trial Workshop -St. Patrick\'s School 2024.xlsx', sheet_name=0))
dataSets.append(pd.read_excel('data/3 days  Robotics Trial Workshop -St. Patrick\'s School 2024.xlsx', sheet_name=1))
dataSets.append(pd.read_excel('data/3 days  Robotics Trial Workshop -St. Patrick\'s School 2024.xlsx', sheet_name=2))
dataSets.append(pd.read_excel('data/3 days  Robotics Trial Workshop -St. Patrick\'s School 2024.xlsx', sheet_name=3))
dataSets.append(pd.read_excel('data/3 hour 2023-2024 Robotics workshop.xlsx', sheet_name=0))
dataSets.append(pd.read_excel('data/3 hour 2023-2024 Robotics workshop.xlsx', sheet_name=1))
dataSets.append(pd.read_excel('data/3 hours Open Invite 22nd April 2024 Robotics.xlsx', sheet_name=0))
dataSets.append(pd.read_excel('data/3 hours Open Invite 22nd April 2024 Robotics.xlsx', sheet_name=1))

#### Getting all Columns

In [4]:
cols = []
for dataSet in dataSets:
  cols.extend(dataSet.columns.str.lower())


###### Removing Trash Columns

In [5]:
cols2 = list(cols)
for X in cols:
  if 'unnamed' in X.lower(): cols2.remove(X)
  if 'column' in X.lower(): cols2.remove(X)

cols = list(cols2)

ignore = [
    'timestamp',
    'response type',
    'start date (utc)',
    'stage date (utc)',
    'submit date (utc)',
    'network id',
    'tags'
    ]
for X in ignore:
  cols.remove(X)

###### Removing Duplicate Columns

In [6]:
cols2 = []
for X in cols:
  if not InList(X, cols2):
    cols2.append(X.lower())
cols = cols2

###### All columns to markdown file

In [7]:
# with open('Columns.md', 'w') as file:
#   text = ''
#   for ind in range(len(cols)):
#     text += '\n'
#     text += f'{ind+1}. '
#     text += cols[ind].lstrip()
#   file.write(text)

#### Making Master DataFrame

##### NLP Functions

In [3]:
from transformers import pipeline
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline(
    task = 'zero-shot-classification',
    model='facebook/bart-large-mnli'
)

qa = pipeline(
    task='question-answering',
    model='deepset/roberta-base-squad2'
)

summarizer = pipeline(
    task='summarization',
    model = 'facebook/bart-large-cnn'
)

def experience(text) -> tuple:
    labels = ['Experienced', 'Unexperienced']
    res = classifier(text, candidate_labels=labels)
    # verdict, score = res['labels'][0], res['scores'][0]
    op = dict(zip(res['labels'], res['scores']))['Experienced']
    # ops = []
    # for d in res:
    #     ops.append(dict(zip(d['labels'], d['scores']))['Experienced'])
    return op

def interest(text: str) -> float:
    labels = ['Super Interested!', 'Uninterested']
    res = classifier(text, candidate_labels=labels)
    op = dict(zip(res['labels'], res['scores']))['Super Interested!']
    return op

def highlightFactor(text):
    res = summarizer(
        text,
        min_length=2,
        max_length=4
    )
    return res

def satisfaction(text: str) -> float:
    labels = ['Satisfied!', 'Unsatisfied!']
    res = classifier(text, candidate_labels=labels)
    op = []
    for d in res:
        op.append(dict(zip(d['labels'], d['scores']))['Satisfied!'])
    return op

def suggestions(text: str):
    res = qa(
        question='What are people\'s opinions or problems with the workshop?',
        context=text,
        min_tokens = 20
    )
    return res['answer']

def expectation(text: str):
    res = qa(
        question='What are the main expectations of the speaker from the workshop?',
        context=text,
        min_tokens = 20
    )
    return res['answer']

def knowledgeGained(text: str) -> float:
    labels = ['has learned a lot', 'has learned nothing']
    res = classifier(f'after the robotics workshop, {text}', candidate_labels=labels)
    op = dict(zip(res['labels'], res['scores']))['has learned a lot']
    return op

Device set to use mps:0
Device set to use mps:0
Device set to use mps:0


In [13]:
interest('When asked, "Do you like robotics", I said, "YESSSSSS"')

0.9950044751167297

##### DB of Questions and Interpreted Features

In [9]:
AllExperienceColumns = [
'what comes to your mind when you hear the word "robots" or robotics"?',
'have you attended a robotics workshop before?',
'how interested are you in attending this workshop at the american center?',
'have you attended a robotics workshop before?',
'what is your understanding of robots?'
'have you done a simple circuit?',
'do you know anything about arduino boards?',
'what do you know about robotics?',
'what do you know about robotics ? (basics, sensors & actuators, arduino, motor controls)',
'what do you know about computer programming/coding/block coding?',
'do you have any experience with coding?',
'have you attended a robotics program or workshop before ?',
' have you worked with simple circuits ?',
' do you know anything about microcontrollers arduino & raspberry pi?',
' what do you know about robotics ? ( basics , sensors & actuators , arduino , motor controls)',
' what is your understanding of robots or robotics?']

AllInterestBeforeColumns = [
    'what comes to your mind when you hear the word "robots" or robotics"?',
	'have you attended a robotics workshop before?',
	'how interested are you in attending this workshop at the american center?',
	'is computer programming/coding/block coding something that interests you?',
	'based on your understanding, is it something you can learn?',
	'if yes , give a brief description. if no would you like to  know about robotics through a workshop', 
	'how interested are you in attending this workshop at american center?',
	'is robotics your passion and how will it align with your future goals?'
]

AllInterestAfterColumns = [
    'would you like to join our whatsapp community, where you will get updates about workshops and also learn cool facts about technology?',
	'will you take stem/computer science as your career?',
	'with all this knowledge do you think you will pursue robotics as your career?',
	'what do you think about robotics after being part of the workshop?',
	'after attending this workshop do you think you will pursue robotics as your career?',
	'do you now have an interest to build your robot?'
]

AllHighlightFactorColumns = [
    'what is that one thing that you have learnt from this workshop and will never forget?',
	'what was the best part of the robotics camp for you',
	'what was one new skill or idea you learned during your project that you found most useful or interesting?',
	'what is one thing you\'ve learned that you will take back with you or that will be useful in your daily life?',
	'can you tell us about your key takeaways?'
]

AllSatisfactionColumns = [
    'on a scale of 10, how would you rate this workshop?',
	'did you feel you had enough help and tools to come up with and build your project idea? what more could we have given you?',
	'did you feel ready for the project part after the first lessons and activities? if not, what basic things could we have taught better?',
	'would you like to join our whatsapp community, where you will get updates about workshops and also learn cool facts about technology?',
	'how was the session overall on a scale of 1-10?',
	'how would you rate the trainers on a scale of 1-10?',
	'how would you rate the trainers?',
	'how was the overall workshop?',
	'on what scale did this workshop satisfy your expectations?',
	'how was your overall workshop experience ?'
]

AllSuggestionColumns = [
	'what suggestions do you have for the trainers to improve the workshop?',
	'what was the hardest thing in the camp? how could we have made it easier or clearer for you?',
	'the camp was 2 hours each day. was this time right? if it was too short/long please tell us why.',
	'thinking about how each day was set up, what changes would you suggest to make each day\'s session better?',
	'what is one main suggestion you have to make the robotics camp even better for the next students?',
	'any comments or suggestions you have regarding the workshop or for the trainers?',
	'what suggestions do you have for the trainers?'
]

AllImprovementColumns = [
    'what topics or activities do you think were missing from the camp, or that you wished we spent more time on?',
	'were there any parts of the camp that were confusing, not important, or not fun? please tell us exactly what.',
	'did you feel ready for the project part after the first lessons and activities? if not, what basic things could we have taught better?',
	'what tools or materials (like websites, specific parts, or other equipment) did youwish you had during the camp?',
	'if you told a friend about this camp, what\'s the main thing you\'d say we could do better?'
    'we\'d love to hear from you!if you enjoyed the robotics camp, please take a moment to leave a google review. your words help more students and schools discover us!',
	'any comments or suggestions? '
]

AllExpectationColumns = [
    '1. what are your expectations from this workshop?',
	'2. what would you like to learn during this workshop?',
	'3. what do you like to gain from this workshop ?'
]

AllKnowledgeGainColumns = [
    'how is your understanding on basic electronics and block coding after attending the workshop?',
	'can you rate your understanding on sensors & motors?',
	'do you think you can design and build your robot in the future with the knowledge you have gained?',
	'do you think you can design and build your own robot in the future with the knowledge you have gained?',
	'with all this knowledge do you think you will pursue robotics as your career?',
	'can you rate your understanding on sensors?',
	'do you now have an understanding of how robots are made ?',
	'do you think you can design and build your own robot in the future using arduino?',
	'based upon the working principle of a robot give us an example (sense - plan - act)',
	'can you rate your understanding on the basics of robotics ?',
	'can you rate your understanding on sensors & actuators ?'
]

##### Adding Columns

In [12]:
data = pd.DataFrame()

In [10]:
names = []
durations = []
modes = []
classes = []
genders = []
workshopNames = []
knowledgeGain = []
interestsBefore = []
interestsAfter = []
expectations = []
highlights = []
satisfactions = []
suggestionsToImprove = []
expectations = []
knowledgeGain = []
experiences = []

durationsPerDataSet = 2*['1 hour'] + ['10 day'] + 2*['2 day'] + 4*['3 day'] + 4*['3 hour']
modePerDataSet = 2*['Online'] + 11*['Offline']
workshopNamePerDataSet = \
2*['1 hour Online intro to Robotics workshop-pilot'] + \
['10 day Summercamp Anna library Feedback'] + \
2*['2 Day learning surveys 2024-2025 American center'] + \
4*['3 days  Robotics Trial Workshop -St. Patrick\'s School 2024'] + \
2*['3 hour 2023-2024 Robotics workshop'] + \
2*['3 hours Open Invite 22nd April 2024 Robotics']



for ind in range(len(dataSets)):
    namesToAdd = list(dataSets[ind].loc[:, columnLike('name', dataSets[ind].columns)])
    count = len(namesToAdd)
    names.extend(namesToAdd)

    durations.extend([durationsPerDataSet[ind]]*count)
    modes.extend([modePerDataSet[ind]]*count)

    workshopNames.extend([workshopNamePerDataSet[ind]]*count)

    classCol = columnLike('class', dataSets[ind].columns)
    if classCol is not None:
        classes.extend(list(dataSets[ind][classCol]))
    else:
        classes.extend([np.nan for _ in range(count)])

    genderCol =  columnLike('gender', dataSets[ind].columns)
    if genderCol is not None:
        genders.extend(list(dataSets[ind][genderCol]))
    else:
        genders.extend([np.nan for _ in range(count)])


    # experienceCols = columnLike(AllExperienceColumns, dataSets[ind].columns, singleOp=False)
    # if experienceCols is not None:
    #     sset = pd.DataFrame(dataSets[ind].loc[:, experienceCols])
    #     for ind in range(len(sset.columns)):
    #         sset.iloc[:, ind] =  sset.iloc[:, ind].apply(lambda x: f'When i was asked, "{list(sset.columns)[ind]}, I said, {x}"').apply(experience)
    #     experiences.extend(list(sset.mean(axis=1)))
    # else:
    #     experiences.extend([np.nan for _ in range(count)])

    # interestBeforeCols = columnLike(AllInterestBeforeColumns, dataSets[ind].columns, singleOp=False)
    # if interestBeforeCols is not None:
    #     sset = pd.DataFrame(dataSets[ind].loc[:, interestBeforeCols])
    #     for ind in range(len(sset.columns)):
    #         sset.iloc[:, ind] =  sset.iloc[:, ind].apply(lambda x: f'When i was asked, "{list(sset.columns)[ind]}, I said, {x}"').apply(interest)
    #     interestsBefore.extend(list(sset.mean(axis=1)))
    # else:
    #     interestsBefore.extend([np.nan for _ in range(count)])

    # interestAfterCols = columnLike(AllInterestAfterColumns, dataSets[ind].columns, singleOp=False)
    # if interestAfterCols is not None:
    #     sset = pd.DataFrame(dataSets[ind].loc[:, interestAfterCols])
    #     for ind in range(len(sset.columns)):
    #         sset.iloc[:, ind] =  sset.iloc[:, ind].apply(lambda x: f'When i was asked, "{list(sset.columns)[ind]}, I said, {x}"').apply(interest)
    #     interestsAfter.extend(list(sset.mean(axis=1)))
    # else:
    #     interestsAfter.extend([np.nan for _ in range(count)])
    
    # suggestionsCols = columnLike(AllSuggestionColumns + AllImprovementColumns, dataSets[ind].columns, singleOp=False)
    # if suggestionsCols is not None:
    #     sset = pd.DataFrame(dataSets[ind].loc[:, suggestionsCols])
    #     for ind in range(len(sset.columns)):
    #         sset.iloc[:, ind] =  sset.iloc[:, ind].apply(lambda x: f'These are the suggestions I have for improvement: {x}"').apply(suggestions)
    #     # print(sset)
    #     combinedSuggestions = pd.Series(['' for _ in range(dataSets[ind].shape[0])])
    #     for col in sset.columns:
    #         combinedSuggestions += sset[col] + ' '
    #     suggestionsToImprove.extend(list(combinedSuggestions))
    # else:
    #     suggestionsToImprove.extend([np.nan for _ in range(count)])

    # highlightCols = columnLike(AllHighlightFactorColumns, dataSets[ind].columns, singleOp=False)
    # if highlightCols is not None:
    #     sset = pd.DataFrame(dataSets[ind].loc[:, highlightCols])
    #     text = pd.Series(['' for _ in range(dataSets[ind].shape[0])])
    #     for ind in range(len(sset.columns)):
    #         text +=  sset.iloc[:, ind] + ' '
    #     #print(text)
    #     # results= highlightFactor(list(text))
    #     highlights.extend(list(text))
    # else:
    #     highlights.extend([np.nan for _ in range(count)])

    satisfactionCols = columnLike(AllSatisfactionColumns, dataSets[ind].columns, singleOp=False)
    if satisfactionCols is not None:
        sset = pd.DataFrame(dataSets[ind].loc[:, satisfactionCols])
        text = pd.Series(['' for _ in range(dataSets[ind].shape[0])])
        for ind in range(len(sset.columns)):
            col = list(sset.columns)[ind]
            text +=  sset.iloc[:, ind].astype('str').apply(lambda x: f'When I was asked, {col}, I said, {x}. ')
    #     print(text)
        results= satisfaction(list(text))
        satisfactions.extend(list(results))
    else:
        satisfactions.extend([np.nan for _ in range(count)])

    # expectationCols = columnLike(AllExpectationColumns, dataSets[ind].columns, singleOp=False)
    # if expectationCols is not None:
    #     sset = pd.DataFrame(dataSets[ind].loc[:, expectationCols])
    #     for ind in range(len(sset.columns)):
    #         sset.iloc[:, ind] =  sset.iloc[:, ind].apply(lambda x: f'These are the expectations I have: {x}"').apply(expectation)
    #     # print(sset)
    #     combinedExpectations = pd.Series(['' for _ in range(dataSets[ind].shape[0])])
    #     for col in sset.columns:
    #         combinedExpectations += sset[col] + ' '
    #     expectations.extend(list(combinedExpectations))
    # else:
    #     expectations.extend([np.nan for _ in range(count)])

    # knowledgeGainColumns = columnLike(AllKnowledgeGainColumns, dataSets[ind].columns, singleOp=False)
    # if knowledgeGainColumns is not None:
    #     sset = pd.DataFrame(dataSets[ind].loc[:, knowledgeGainColumns])
    #     for ind in range(len(sset.columns)):
    #         sset.iloc[:, ind] =  sset.iloc[:, ind].apply(lambda x: f'When i was asked, "{list(sset.columns)[ind]}, I said, {x}"').apply(knowledgeGained)
    #     knowledgeGain.extend(list(sset.mean(axis=1)))
    # else:
    #     knowledgeGain.extend([np.nan for _ in range(count)])

In [13]:
data['name'] = pd.Series(names)
data['gender'] = pd.Series(genders)
data['class'] = pd.Series(classes)
data['mode'] = pd.Series(modes)
data['workshop_name'] = pd.Series(workshopNames)
data['workshop_duration'] = pd.Series(durations)
data['interest_before'] = pd.Series(interestsBefore)
data['interest_after'] = pd.Series(interestsAfter)
data['highlights'] = pd.Series(highlights)
data['suggestions'] = pd.Series(suggestionsToImprove)
data['satisfaction'] = pd.Series(satisfactions)
data['expectations'] = pd.Series(expectations)
data['prior_experience'] = pd.Series(experiences)
data['knowledge_gain'] = pd.Series(knowledgeGain)

##### Saving the CSV

In [14]:
data.to_csv('satisfaction.csv', index=False)

In [ ]:
data.knowledge_gain.isna().sum()

np.int64(872)